In [ ]:
import pandas as pd
from google.colab import drive
import plotly.graph_objects as go
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional, GRU
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error, mean_squared_error

from sklearn.preprocessing import LabelEncoder

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dfr = pd.read_excel(r'/content/drive/My Drive/NVDA.xlsx')

In [ ]:
df=dfr.copy()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
full_date_range = pd.date_range(start=df['Date'].min(), end=df['Date'].max(), freq='D')
df.set_index('Date', inplace=True)
df = df.reindex(full_date_range)

df.reset_index(inplace=True)
df.rename(columns={'index': 'Date'}, inplace=True)

In [ ]:
for i in df.columns[1:]:
  df[i] = df[i].interpolate()

In [ ]:
df.set_index('Date', inplace=True)

In [ ]:
def MAPE(actual,prediction):
    mape = mean_absolute_percentage_error(actual,prediction)
    return mape

def R2(actual,prediction):
    r2= r2_score(actual,prediction)
    return r2

def MAE(actual,prediction):
    mae= mean_absolute_error(actual,prediction)
    return mae

def RMSE(actual,prediction):
    rmse= np.sqrt(mean_squared_error(actual,prediction))
    return rmse


In [ ]:
data = df.filter(['Close'])
dataset = data.values
training_data_len = int(np.ceil( len(dataset) * .90 ))


In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)


In [ ]:
train_data = scaled_data[0:int(training_data_len), :]
# Split the data into x_train and y_train data sets
x_train = []
y_train = []

for i in range(90, len(train_data)):
    x_train.append(train_data[i-90:i, 0])
    y_train.append(train_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


In [ ]:
def rnn_model(mod):
  model = Sequential()

  if mod == 'GRU':
      model = Sequential()
      model.add(GRU(128, return_sequences=True, input_shape= (x_train.shape[1], 1)))
      model.add(GRU(64, return_sequences=False))

  elif mod == 'LSTM':
      model = Sequential()
      model.add(LSTM(128, return_sequences=True, input_shape= (x_train.shape[1], 1)))
      model.add(LSTM(64, return_sequences=False))

  model.add(Dense(25))
  model.add(Dense(1))
  model.compile(optimizer='adam', loss='mean_squared_error')

  model.fit(x_train, y_train, batch_size=1, epochs=1)


  test_data = scaled_data[training_data_len - 90: , :]
  x_test = []
  y_test = dataset[training_data_len:, :]
  for i in range(90, len(test_data)):
      x_test.append(test_data[i-90:i, 0])

  x_test = np.array(x_test)

  x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))

  predictions = model.predict(x_test)
  predictions = scaler.inverse_transform(predictions)

  rmse = RMSE(y_test,predictions)
  mape =MAPE(y_test,predictions)
  mae = MAE(y_test,predictions)
  r2 = R2(y_test,predictions)

  print("RMSE:" ,rmse, "\nMAPE:" ,mape, "\nMAE:" ,mae, "\nR2:" ,r2)

  train = data[:training_data_len]
  valid = data[training_data_len:]
  valid['Predictions'] = predictions
  # Visualize the data
  trace_train = go.Scatter(
      x=train.index,
      y=train['Close'],
      mode='lines',
      name='Train',
      line=dict(color='blue')
  )

  trace_valid_close = go.Scatter(
      x=valid.index,
      y=valid['Close'],
      mode='lines',
      name='Val',
      line=dict(color='blue')
  )

  trace_valid_pred = go.Scatter(
      x=valid.index,
      y=valid['Predictions'],
      mode='lines',
      name='Predictions',
      line=dict(color='red', dash='dash')
  )

  layout = go.Layout(
      title='Model',
      xaxis_title='Date',
      yaxis_title='Close Price USD ($)',
      legend=dict(x=0, y=-0.2)
  )

  fig = go.Figure(data=[trace_train, trace_valid_close, trace_valid_pred], layout=layout)

  fig.show()







In [ ]:
rnn_model('LSTM')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



1556/1556 ━━━━━━━━━━━━━━━━━━━━ 122s 77ms/step - loss: 9.4137e-04
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step
RMSE: 4.801333014608689 
MAPE: 0.03612917040352978 
MAE: 3.702531530839522 
R2: 0.9263129618655278


<ipython-input-131-95580e259521>:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
rnn_model('GRU')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



1556/1556 ━━━━━━━━━━━━━━━━━━━━ 122s 77ms/step - loss: 0.0016
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 96ms/step
RMSE: 3.8306927232820005 
MAPE: 0.02979606545498783 
MAE: 3.0180748899855843 
R2: 0.9530946829959853


<ipython-input-131-95580e259521>:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

